# Solution
First, we create a CuttingStock class which stores the data

In [55]:
from gurobipy import GRB, Model, quicksum
import random as r

In [56]:
class CuttingStockProblem:

    def __init__(self,width_large_rolls:float, width_small_rolls:list, demand_small_rolls:list):
        self.width_large_rolls = width_large_rolls
        self.width_small_rolls = width_small_rolls
        self.demand = demand_small_rolls

    def get_max_number_of_large_rolls(self):
        '''
        Returns an upper bound on the number of large rolls used.
        It assumes that each small roll is cut from a different large roll,
        therefore, the upper bound is equal to the sum of the demands.
        :return:int
        '''
        return sum(self.demand)

    def get_n_small_roll_types(self):
        return len(self.width_small_rolls)

    def get_feasible_patterns(self):

        # First we calculate how many of each width we can cut from a large roll
        max_rolls = [int((self.width_large_rolls/self.width_small_rolls[i])//1) for i in range(self.get_n_small_roll_types())]

        # Now, we generate all possible patterns, and then we
        # check which ones are feasible. All
        # possible patterns are given by the Cartesian product of
        # the integers up to max_rolls.
        # That is, if we can cut
        # -- up to 2 of width w1, that is 0, 1 or 2
        # -- up to 2 of width w2, that is 0, 2 or 2
        # -- up to 3 of width w3, that is 0, 1, 2 or 3
        # all the possible patters are given by the Cartesian product
        # of the vectors[0, 1, 2]x[0, 1, 2]x[0, 1, 2, 3]
        vectors = {}
        for i in range(self.get_n_small_roll_types()):
            vectors[i] = [j for j in range(max_rolls[i] + 1)]

        patterns = None
        for i in vectors:
            if patterns == None:
                patterns = self.cartesian_product_of_one(vectors[i])
            else:
                patterns = self.cartesian_product_of_two(patterns,vectors[i])

        # Now, of the cartesian products, we discard the elements which violate the large roll width
        infeasible_patterns = []
        for p in patterns:
            cut_width = sum([self.width_small_rolls[i] * p[i] for i in range(self.get_n_small_roll_types())])
            if cut_width > self.width_large_rolls:
                infeasible_patterns.append(p)
        for p in infeasible_patterns:
            patterns.remove(p)

        return patterns

    def cartesian_product_of_one(self,a:list):
        return [(i,) for i in a]

    def cartesian_product_of_two(self,a:list,b:list) -> list:
        elements = []
        for i in a:
            for j in b:
                elements.append(i+(j,))
        return(elements)


Then we generate an instance of the problem

In [80]:
width_large_rolls = 5
width_small_rolls = [2.1,1.8,1.5]
demand_small_rolls = [9,12,19]

p = CuttingStockProblem(width_large_rolls,width_small_rolls,demand_small_rolls)


We create the original model

In [85]:
class CuttingStockModel1:

    def __init__(self, p:CuttingStockProblem):
        self.p = p
        self.m = Model('CS1')

        # Decision variables
        n_large_rolls = self.p.get_max_number_of_large_rolls()
        n_small_roll_types = len(self.p.demand)
        y = self.m.addVars([i for i in range(n_large_rolls)],vtype=GRB.BINARY,name="y")
        z = self.m.addVars([j for j in range(n_small_roll_types)],[i for i in range(n_large_rolls)],lb= 0, ub= GRB.INFINITY, vtype=GRB.INTEGER,name="x")

        # Objective function
        self.m.setObjective(y.sum('*'))

        # Constraints
        # Note that the argument passed to quicksum is a list, and the list is built using comprehension
        self.m.addConstrs(quicksum([self.p.width_small_rolls[i] * z[i,j] for i in range(n_small_roll_types)]) <= self.p.width_large_rolls * y[j] for j in range(n_large_rolls))
        self.m.addConstrs(z.sum(i,'*') >= self.p.demand[i] for i in range(n_small_roll_types))

    def solve(self):
        self.m.Params.timeLimit = 30.0
        self.m.optimize()

    def print_objective(self):
        print("Objective ", self.m.objVal)
    def print_solution(self):
        for v in self.m.getVars():
            print("%s %g" % (v.varName, v.x))
    def print_runtime(self):
        print("Solved in %g seconds" % self.m.Runtime)
    def print_gap(self):
        print("Optimality gap %g percent" % (100 * self.m.MIPGap))
    def print_bound(self):
        print("Best bound %g" % self.m.ObjBound)

In [86]:
m1 = CuttingStockModel1(p)
m1.solve()
m1.print_objective()
m1.print_gap()
m1.print_bound()

Set parameter TimeLimit to value 30
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 128 physical cores, 128 logical processors, using up to 32 threads
Academic license - for non-commercial use only - registered to gp@math.ku.dk
Optimize a model with 43 rows, 160 columns and 280 nonzeros
Model fingerprint: 0x19c5d7cf
Variable types: 0 continuous, 160 integer (40 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e+00, 2e+01]
Found heuristic solution: objective 18.0000000
Presolve time: 0.00s
Presolved: 43 rows, 160 columns, 280 nonzeros
Variable types: 0 continuous, 160 integer (40 binary)

Root relaxation: objective 1.437500e+01, 95 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   14.37500    0   17   1

We create the second model

In [87]:
class CuttingStockModel2:

    def __init__(self, p:CuttingStockProblem):
        self.p = p
        self.m = Model('CS2')

        # Decision variables
        print("Getting patterns..")
        self.patterns = self.p.get_feasible_patterns()
        n_patterns = len(self.patterns)
        print("Got %g patterns" % n_patterns)

        x = self.m.addVars(n_patterns,lb= 0, ub= GRB.INFINITY, vtype=GRB.INTEGER,name="x")

        # Objective function
        self.m.setObjective(x.sum('*'))

        # Constraints
        # Note that the argument passed to quicksum is a list, and the list is built using comprehension
        self.m.addConstrs(quicksum([self.patterns[q][i] * x[q] for q in range(n_patterns)]) >= self.p.demand[i]  for i in range(len(p.demand)))

    def solve(self):
        self.m.Params.timeLimit = 30.0
        self.m.optimize()


    def print_objective(self):
        print("Objective ", self.m.objVal)
    def print_solution(self):
        for v in self.m.getVars():
            print("%s %g" % (v.varName, v.x))
    def print_runtime(self):
        print("Solved in %g seconds" % self.m.Runtime)
    def print_gap(self):
        print("Optimality gap %g percent" % (100* self.m.MIPGap))
    def print_bound(self):
        print("Best bound %g" % self.m.ObjBound)

In [88]:
m2 = CuttingStockModel2(p)
m2.solve()
m2.print_objective()
m2.print_gap()
m2.print_bound()

Getting patterns..
Got 12 patterns
Set parameter TimeLimit to value 30
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 128 physical cores, 128 logical processors, using up to 32 threads
Academic license - for non-commercial use only - registered to gp@math.ku.dk
Optimize a model with 3 rows, 12 columns and 15 nonzeros
Model fingerprint: 0x36e3dd90
Variable types: 0 continuous, 12 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e+00, 2e+01]
Found heuristic solution: objective 31.0000000
Presolve removed 0 rows and 6 columns
Presolve time: 0.00s
Presolved: 3 rows, 6 columns, 9 nonzeros
Variable types: 0 continuous, 6 integer (0 binary)

Root relaxation: objective 1.525000e+01, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd 

In [89]:
width_large_rolls = 11
n_small_roll_types = 8
r.seed(1)

width_small_rolls = [(2 + (r.random() * 2)) for i in range(n_small_roll_types)]
demand_small_rolls = [r.randint(10,20)  for i in range(n_small_roll_types)]

p = CuttingStockProblem(width_large_rolls,width_small_rolls,demand_small_rolls)
#p.get_feasible_patterns()

In [90]:
m1 = CuttingStockModel1(p)
m1.solve()
m1.print_objective()
m1.print_gap()
m1.print_bound()
m1.print_runtime()

Set parameter TimeLimit to value 30
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 128 physical cores, 128 logical processors, using up to 32 threads
Academic license - for non-commercial use only - registered to gp@math.ku.dk
Optimize a model with 124 rows, 1044 columns and 1972 nonzeros
Model fingerprint: 0xc40a5e2a
Variable types: 0 continuous, 1044 integer (116 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 2e+01]
Presolve time: 0.00s
Presolved: 124 rows, 1044 columns, 1972 nonzeros
Variable types: 0 continuous, 1044 integer (116 binary)
Found heuristic solution: objective 35.0000000

Root relaxation: objective 3.272632e+01, 255 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   32.72632   

In [92]:
m2 = CuttingStockModel2(p)
m2.solve()
m2.print_objective()
m2.print_bound()
m2.print_gap()
m2.print_runtime()

Getting patterns..
Got 203 patterns
Set parameter TimeLimit to value 30
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 128 physical cores, 128 logical processors, using up to 32 threads
Academic license - for non-commercial use only - registered to gp@math.ku.dk
Optimize a model with 8 rows, 203 columns and 458 nonzeros
Model fingerprint: 0xb9087079
Variable types: 0 continuous, 203 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+01]
Found heuristic solution: objective 105.0000000
Presolve removed 0 rows and 78 columns
Presolve time: 0.00s
Presolved: 8 rows, 125 columns, 311 nonzeros
Variable types: 0 continuous, 125 integer (0 binary)

Root relaxation: objective 3.342424e+01, 10 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbe